In [2]:
!pip install -q transformers datasets

#### Example from https://huggingface.co/tasks/zero-shot-classification

In [3]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli")
sequence_to_classify = "Angela Merkel is a politician in Germany and leader of the CDU"
candidate_labels = ["politics", "economy", "entertainment", "environment"]
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)

{'sequence': 'Angela Merkel is a politician in Germany and leader of the CDU', 'labels': ['politics', 'environment', 'economy', 'entertainment'], 'scores': [0.9967535734176636, 0.0014776551397517323, 0.0012541451724246144, 0.000514575804118067]}


### Deployment as an endpoint on SageMaker

In [4]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

In [5]:
role = sagemaker.get_execution_role()

In [6]:
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli',
    'HF_TASK':'zero-shot-classification'
}
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    env=hub,
    role=role, 
)

In [7]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type="ml.g4dn.xlarge" # ec2 instance type
)


----------!

In [36]:
text = {
    "inputs": "one day I will see the world", 
    "parameters": {
        "candidate_labels": ['travel', 'cooking', 'dancing']
    }
}

In [35]:
predictor.predict(text)

{'sequence': 'one day I will see the world',
 'labels': ['travel', 'cooking', 'dancing'],
 'scores': [0.9943726658821106, 0.0028813271783292294, 0.0027459932025521994]}

In [86]:
# predictor.delete_endpoint()

## Inference with SDK Boto3

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv("./dataset/sem_eval_2018_task_1_test.csv")

In [46]:
label_name = [label for label in df.columns if label not in ['ID', 'Tweet']]
label_name

['anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'love',
 'optimism',
 'pessimism',
 'sadness',
 'surprise',
 'trust']

In [47]:
label_index = df.iloc[:,2:].astype(int)
label_index.head(5)

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,1,1,0,0,0,0,1,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,1,0,0


In [48]:
text_tweet = df.loc[:,["Tweet"]].values.tolist()

In [49]:
text_tweet[0]

['@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell']

In [50]:
import boto3
import json

In [51]:
runtime = boto3.client("sagemaker-runtime")

In [37]:
endpoint_name = "huggingface-pytorch-inference-2023-03-16-07-08-39-697"

In [54]:
input_json = {
    "inputs": text_tweet[0], 
    "parameters": {
        "candidate_labels": label_name
    }
}

In [55]:
input_json

{'inputs': ['@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell'],
 'parameters': {'candidate_labels': ['anger',
   'anticipation',
   'disgust',
   'fear',
   'joy',
   'love',
   'optimism',
   'pessimism',
   'sadness',
   'surprise',
   'trust']}}

In [57]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(input_json),
    ContentType="application/json",
)

print(response["Body"].read())

b'[{"sequence":"@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell","labels":["optimism","anticipation","fear","anger","surprise","trust","disgust","pessimism","joy","love","sadness"],"scores":[0.5757121443748474,0.32633233070373535,0.019478188827633858,0.018926115706562996,0.01643146201968193,0.012336432933807373,0.009881631471216679,0.009710927493870258,0.0051020849496126175,0.003098501358181238,0.002990258624777198]}]'
